In [1]:
import datetime
from glob import glob
import json
import os
import sys

import gym
import numpy as np
from PIL import Image
import resampy
from stable_baselines3.common.noise import NormalActionNoise
import torch
from torchvision import transforms
from transformers import Wav2Vec2ForCTC, Wav2Vec2Processor
import yaml

from custom_policy2 import CustomTD3PolicyCNN, CustomDDPG


2023-02-28 17:03:03.008812: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-02-28 17:03:05.605911: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory
2023-02-28 17:03:05.606077: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer_plugin.so.7'; dlerror: libnvinfer_plugin.so.7: cannot open shared object file: No such file or directory
2023-02-28 17:03:05.606083: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Cannot dlopen some TensorRT libraries. If you would like to use Nv

In [2]:
sys.path.append("../..")
import hifigan
from hifigan.env import AttrDict
from hifigan.models import Generator

sys.path.append("../U2S")
from hparams import create_hparams
from train import load_model
from text import text_to_sequence


sys.path.append("../I2U")
from models import TransformerSentenceLM, TransformerConditionedLM
from models_modified import TransformerSentenceLM_FixedImg

# config path需要更改
with open('../../config.yml') as yml:
    config = yaml.safe_load(yml)


device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [3]:
# I2U
word_map_path="/net/papilio/storage2/yhaoyuan/transformer_I2S/data/processed/SpokenCOCO_LibriSpeech/WORDMAP_coco_1_cap_per_img_1_min_word_freq.json"
# Load word map (word2ix)
with open(word_map_path) as j:
    word_map = json.load(j)
rev_word_map = {v: k for k, v in word_map.items()}  # ix2word
special_words = {"<unk>", "<start>", "<end>", "<pad>"}

# I2U
# checkpoint_path = "../../saved_model/I2U/VC_5_captions/Trial_1/"
checkpoint_path = "../../saved_model/I2U/VC_5_captions_224/fixed_img_1024_sentence_8/"
# checkpoint_path = "../../saved_model/I2U/VC_5_captions_224/"

with open(checkpoint_path + 'config_sentence.yml', 'r') as yml:
    model_config = yaml.safe_load(yml)

checkpoint = checkpoint_path + f'bleu-4_BEST_checkpoint_coco_{str(model_config["i2u"]["captions_per_image"])}_cap_per_img_{str(model_config["i2u"]["min_word_freq"])}_min_word_freq_gpu.pth.tar'
dir_name = model_config["i2u"]["dir_name"]
model_params = model_config["i2u"]["model_params"]
train_params = model_config["i2u"]["train_params"]
img_refine_params = model_config["i2u"]["refine_encoder_params"]

data_folder = f'../../data/processed/{dir_name}/'  # folder with data files saved by create_input_files.py
# data_name = 'coco_4_cap_per_img_5_min_word_freq'  # base name shared by data files
#data_name = f'coco_{str(config["i2u"]["captions_per_image"])}_cap_per_img_{str(config["i2u"]["min_word_freq"])}_min_word_freq'  # base name shared by data files
data_name = f'coco_{str(model_config["i2u"]["captions_per_image"])}_cap_per_img_{str(model_config["i2u"]["min_word_freq"])}_min_word_freq'  # base name shared by data files
model_params['vocab_size'] = len(word_map)
model_params['refine_encoder_params'] = img_refine_params
model = TransformerSentenceLM_FixedImg(**model_params)
# optimizer = getattr(torch.optim, train_params["optimizer"])(model.parameters(), lr=train_params["lr"])
# model, optimizer, start_epoch, best_bleu4, best_accuracy = load_checkpoint(checkpoint, model, optimizer, device)
model.load_state_dict(torch.load(checkpoint)["model_state_dict"])
model.eval()
model.to(device)

# model_path = config["i2u"]["model"]
# model_config = model_path[:-len(model_path.split("/")[-1])] + "config.yml"
# with open(model_config) as yml:
#     model_config = yaml.safe_load(yml)
# model_params = model_config["i2u"]["sentence_model_params"]
# model_params['vocab_size'] = len(word_map)
# sentence_encoder = TransformerSentenceLM(**model_params)
# trained_model = torch.load(model_path)
# state_dict = trained_model["model_state_dict"]
# sentence_encoder.load_state_dict(state_dict)
# sentence_encoder.eval()
# sentence_encoder.to(device)

# --------------------------------------------------------------------------------

# U2S
# /net/papilio/storage2/yhaoyuan/LAbyLM/dataprep/RL/image2speech_inference.ipynb

# tacotron2
hparams = create_hparams()
hparams.sampling_rate = 22050
checkpoint_path = config["u2s"]["tacotron2"]
# checkpoint_path = "/net/papilio/storage2/yhaoyuan/LAbyLM/model/U2S/outdir_origin/checkpoint_67000"
tacotron2_model = load_model(hparams)
tacotron2_model.load_state_dict(torch.load(checkpoint_path)['state_dict'])
tacotron2_model.cuda().eval()

# --------------------------------------------------------------------------------

# HiFi-GAN
# /net/papilio/storage2/yhaoyuan/LAbyLM/dataprep/RL/image2speech_inference.ipynb

checkpoint_file = config["u2s"]['hifigan']
config_file = os.path.join(os.path.split(checkpoint_file)[0], 'config.json')
with open(config_file) as f:
        data = f.read()

global h
json_config = json.loads(data)
h = AttrDict(json_config)
generator = Generator(h).to(device)
assert os.path.isfile(checkpoint_file)
checkpoint_dict = torch.load(checkpoint_file, map_location=device)
generator.load_state_dict(checkpoint_dict['generator'])
generator.eval()
generator.remove_weight_norm()

# --------------------------------------------------------------------------------

# S2T
processor = Wav2Vec2Processor.from_pretrained(config["asr"]["model_path"])
asr_model = Wav2Vec2ForCTC.from_pretrained(config["asr"]["model_path"]).to(device)

# --------------------------------------------------------------------------------

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


Removing weight norm...


In [4]:
sys.path.append("..")
from I2U.datasets import *

In [5]:
data_folder = f'../../data/processed/VC_5_captions_224/'  # folder with data files saved by create_input_files.py
data_name = f'coco_{str(config["i2u"]["captions_per_image"])}_cap_per_img_{str(config["i2u"]["min_word_freq"])}_min_word_freq'  # base name shared by data files


In [6]:
normalize = transforms.Normalize(mean=[0.485, 0.456, 0.406],
                                     std=[0.229, 0.224, 0.225])
val_loader = torch.utils.data.DataLoader(
CaptionDataset_transformer(data_folder, data_name, 'VAL', transform=transforms.Compose([normalize])),
batch_size=1, shuffle=True, num_workers=10, pin_memory=True)
# val_loader = torch.utils.data.DataLoader(
# CaptionDataset_transformer(data_folder, data_name, 'VAL', transform=None),
# batch_size=1, shuffle=True, num_workers=10, pin_memory=True)

In [7]:
for i, (imgs, seq, caplens, seq_padding_mask, _, _) in enumerate(val_loader):
    imgs = imgs.to(device)
    seq = seq.to(device)
    caplens = caplens.to(device)
    caplens = caplens.squeeze()
    seq_padding_mask = seq_padding_mask.to(device)

    seq_len = caplens

    # x = sentence_encoder.embed(seq)
    # x = sentence_encoder.pos_encoder(x)
    # z = sentence_encoder.sentence_encoder(x, src_key_padding_mask = seq_padding_mask)
    # z = z * seq_padding_mask.logical_not().unsqueeze(2)
    # z = z.sum(dim = 1)/ seq_len
    # mu = sentence_encoder.mu(z)  # (batch, sentence_embed)

    # imgs, gx = sentence_encoder.image_encoder(imgs)
    break

In [8]:
seq_gt = [int(unit) for unit in seq.squeeze(0) if unit != 0]

In [9]:
mu = model.get_mu(seq, seq_len.unsqueeze(0), seq_padding_mask)

In [11]:
imgs.shape

torch.Size([1, 3, 224, 224])

In [89]:
# imgs, gx = model.image_encoder(imgs)
# action_sentence = torch.cat([imgs.flatten(), mu.flatten()]).unsqueeze(0)
# action = imgs.flatten().unsqueeze(0)

In [12]:
seq = model.decode(image=imgs, start_unit=word_map["<start>"], end_unit=word_map["<end>"], max_len=130, beam_size=10)
# seq = model.decode(action=action_sentence, start_unit=word_map["<start>"], end_unit=word_map["<end>"], max_len=130, beam_size=10)
# seq = model.decode(start_unit=word_map["<start>"], end_unit=word_map["<end>"], action=action, max_len=130, beam_size=10)

In [13]:
seq

[1015,
 490,
 852,
 960,
 409,
 858,
 763,
 764,
 797,
 319,
 49,
 130,
 461,
 157,
 568,
 201,
 488,
 86,
 831,
 884,
 10,
 303,
 249,
 712,
 450,
 866,
 451,
 84,
 153,
 631,
 351,
 267,
 303,
 249,
 23,
 205,
 563,
 292,
 711,
 666,
 588,
 852,
 715,
 707,
 127,
 883,
 873,
 183,
 347,
 297,
 265,
 751,
 702,
 742,
 453,
 84,
 338,
 709,
 108,
 110,
 350,
 111,
 4,
 5,
 165,
 490,
 1016]

In [95]:
def u2s2t(seq):
    words = [rev_word_map[ind] for ind in seq if rev_word_map[ind] not in special_words]
    print(words)
    sequence = np.array(text_to_sequence(' '.join(words), ['english_cleaners']))[None, :]
    sequence = torch.autograd.Variable(torch.from_numpy(sequence)).cuda().long()
    _, mel_outputs_postnet, _, _ = tacotron2_model.inference(sequence)
    with torch.no_grad():
        x = mel_outputs_postnet.squeeze().to(device)
        y_g_hat = generator(mel_outputs_postnet)
        audio = y_g_hat.squeeze()
        
        # audio = audio * 32768.0
        # audio = audio.cpu().numpy().astype('int16')

        audio = audio.cpu().numpy().astype(np.float64)

        import IPython.display as ipd
        display(ipd.Audio(audio, rate=22050))
        input_values = processor(audio, sampling_rate=16000, return_tensors="pt").input_values.float()
        logits = asr_model(input_values.to(device)).logits
        predicted_ids = torch.argmax(logits, dim=-1)
        transcription = processor.decode(predicted_ids[0])
        print(transcription)

In [96]:
u2s2t(seq_gt)


['808', '680', '279', '310', '874', '56', '807', '841', '100', '384', '985', '284', '60', '29', '0', '886', '73', '255', '848', '356', '896', '354', '636', '461', '985', '909', '154', '863', '927', '861', '504', '965', '330', '1016', '394', '287', '375', '35', '100', '384', '985', '909', '402', '681', '777', '271', '329', '963', '950', '578', '42', '145', '71', '229', '479', '6', '865']


three grapes in a brown background


In [97]:
u2s2t(seq)

['865', '100', '634', '914', '247', '427', '585', '677', '116', '644', '967', '997', '500', '720', '874', '56', '503', '807', '714', '455', '384', '985', '284', '531', '60', '807', '141', '775', '716', '398', '848', '356', '896', '588', '636', '461', '985', '1020', '154', '863', '734', '566', '724', '27', '360', '394', '253', '375', '264', '518', '985', '470', '1020', '1000', '681', '901', '939', '358', '950', '311', '709', '410', '71', '479', '229', '479', '803']


there are three grapes in a brown background


In [10]:
from random import shuffle
a = [1,2,3,4,5] 
b = [1,2,3,4,5] 
c = list(zip(a,b))
c_shuffle = shuffle(c)

In [15]:
aa, bb = zip(*c)

In [16]:
aa

(2, 4, 1, 3, 5)

In [17]:
bb

(2, 4, 1, 3, 5)

In [18]:
epoch = 4
(epoch+1)%5 != 0

False

In [25]:

from glob import glob
model_path = "/net/papilio/storage2/yhaoyuan/transformer_I2S/saved_model/I2U/origin_5_captions_256/baseline_lr-3_no_LM"

In [30]:
model_checkpoint_list = glob(model_path+f"/[0-9]*.tar")
sorted_cp = sorted(model_checkpoint_list, key=lambda name: int(name.split("/")[-1].split("_")[0]))

In [31]:
sorted_cp

['/net/papilio/storage2/yhaoyuan/transformer_I2S/saved_model/I2U/origin_5_captions_256/baseline_lr-3_no_LM/0_checkpoint_coco_5_cap_per_img_1_min_word_freq_gpu.pth.tar',
 '/net/papilio/storage2/yhaoyuan/transformer_I2S/saved_model/I2U/origin_5_captions_256/baseline_lr-3_no_LM/4_checkpoint_coco_5_cap_per_img_1_min_word_freq_gpu.pth.tar',
 '/net/papilio/storage2/yhaoyuan/transformer_I2S/saved_model/I2U/origin_5_captions_256/baseline_lr-3_no_LM/9_checkpoint_coco_5_cap_per_img_1_min_word_freq_gpu.pth.tar',
 '/net/papilio/storage2/yhaoyuan/transformer_I2S/saved_model/I2U/origin_5_captions_256/baseline_lr-3_no_LM/14_checkpoint_coco_5_cap_per_img_1_min_word_freq_gpu.pth.tar',
 '/net/papilio/storage2/yhaoyuan/transformer_I2S/saved_model/I2U/origin_5_captions_256/baseline_lr-3_no_LM/19_checkpoint_coco_5_cap_per_img_1_min_word_freq_gpu.pth.tar',
 '/net/papilio/storage2/yhaoyuan/transformer_I2S/saved_model/I2U/origin_5_captions_256/baseline_lr-3_no_LM/24_checkpoint_coco_5_cap_per_img_1_min_word_fr